In [10]:
#nesse programa irei salvar os dados para edita-los em formas de gráficos em outro programa,
#chamado OriginLAbPro
#chamar as funções necessárias
import re
import pandas as pd
import numpy as np
import string 
from  more_itertools import unique_everseen   

In [11]:
#irei ler os arquvivos necessários para extrair informações sobre os filmes e usuários
#declarar a função para tokenize (essa função vai retirar as pontuações e separar as palavras) 
#o texto e declarar uma lista com os genêros dos filmes


data_table1 = pd.read_csv("ratings.csv", usecols = ["userId","movieId", "rating"] )
movies = pd.read_csv("movies.csv", usecols = ["movieId","title", "genres"])

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

genre_all=['action', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama',
          'fantasy', 'film', 'horror', 'musical', 'mystery', 'romance', 'sci', 'thriller',
          'war', 'western', 'no']

In [12]:
#nessa parte inicial vou investigar quantos filmes foram avaliados por cada usuário e em seguida
#irei agrupar os usuários que avaliaram a mesma quantidade de avaliações
user_rating_amount=(data_table1.groupby('userId').userId.count()).values
user_rating_amount_unique=list(unique_everseen(user_rating_amount))
user_rating_amount=pd.DataFrame({'user_rating_amount': user_rating_amount})

histogram_1=(user_rating_amount.groupby('user_rating_amount').user_rating_amount.count()).values
final_histogram = np.column_stack([user_rating_amount_unique, histogram_1])

#salvar os dados
np.savetxt('filmes_avaliados_por_usuário.dat', final_histogram) 

#vou investigar a quantidade de vezes cada filmes foi avaliado e em seguida
#irei agrupar os filmes  com a mesma quantidade de avaliações
film_rating_amount=(data_table1.groupby('movieId').userId.count()).values
film_rating_amount_unique=list(unique_everseen(film_rating_amount))

film_rating_amount=pd.DataFrame({'film_rating_amount': film_rating_amount})
histogram_2=(film_rating_amount.groupby('film_rating_amount').film_rating_amount.count()).values

final_histogram_2 = np.column_stack([film_rating_amount_unique, histogram_2])
#salvar os dados
np.savetxt('filmes_avaliados.dat', final_histogram_2)

In [13]:
#filmes avaliados por ano de lançamento (inicialmente estou considerando de 1950 até 2015, mas 
#esse período ser modificado)
year = list(range(1950, 2016)) 

#organizar o texto 
movies['tokenized'] = movies['title'].apply(lambda x: tokenize(x.lower()))

#construir um data frame apenas com o texto que descreve o titulo do filme
year_movies=pd.DataFrame(list(movies.iloc[:, 3].values))

quantity_filme_year = []
column=np.shape(year_movies)[1]

#procurar o ano de lançamento do filme no dataframe anterior, como no título pode haver um ano 
# (exemplo 2001 uma odisséia no espaço), eu vou considerar apenas anos que aparecem em ultimo
for i in range (0, len(year)):
    quantity=0
    for j in range (1, column):
        if j < (column-2):
            selected = year_movies.loc[(year_movies[j]==str(year[i])) & (year_movies[j+2].isnull())]
        if j == (column-2):
            selected = year_movies.loc[(year_movies[j]==str(year[i]))]
        quantity=quantity+(np.shape(selected)[0])
    
    quantity_filme_year.append(quantity)
#salvar os dados    
np.savetxt('quantity_filme_year.dat', quantity_filme_year) 

In [14]:
#irei conferir a quantidade de cada gênero entre os filmes avaliados
movies['tokenized'] = movies['genres'].apply(lambda x: tokenize(x.lower()))

#organizar o texto que descreve o gênero do filme
movie_genre=pd.DataFrame(list(movies.iloc[:, 3]. values))

quantity_genre = []
column = np.shape(movie_genre)[1]

#contar os filmes de cada gênero (o filme que tiver n gêneros será contabilizado n vezes)
for i in range (0, len(genre_all)):
    quantity = 0
    for j in range (0, column):
        selected = movie_genre.loc[movie_genre[j]==genre_all[i]] 
        quantity = quantity+np.shape(selected)[0]
    quantity_genre.append(quantity)

#salvar os dados
np.savetxt('genre_quantity.dat', quantity_genre)       

#irei conferir a quantidade de cada filme com apenas um gênero
quantity_genre_unic = []
for i in range (0, len(genre_all)):
    if not (i == 8) or (i == 13):
        if not (i==18):
            selected = movie_genre.loc[ (movie_genre[0] == genre_all[i]) & (movie_genre[1].isnull() )]
                
    if (i == 8) or (i == 13):
        selected = movie_genre.loc[ (movie_genre[0] == genre_all[i]) & (movie_genre[2].isnull() )]                            

    quantity_genre_unic.append(np.shape(selected)[0])    
np.savetxt('genre_unic_quantity.dat', quantity_genre_unic)       

In [ ]:
#Irei criar dois grupos o primeiro com filmes que receberam pelo menos uma nota acima de 3.5
#e o segundo grupo com filmes que receberam pelo menos uma nota menor que 3.5
#vale dizer que o mesmo filme pode estar em ambos grupos, já que os usuários tem gostos diferentes.
#No final irei dividir cada grupo pelo gênero do filme

for i in range (0, 2):
#selecionar filmes que tenham notas acima de 3.5 ou abaixo de 3.5    
        if i == 0:
            selected_film = (data_table1.loc[data_table1['rating'] > 3.5])
        if i == 1:
            selected_film = (data_table1.loc[data_table1['rating'] < 3.5])    
#ler as caracteristicas dos filmes selecionados
        ids=list(unique_everseen(selected_film.iloc[:, 1].values))

        movie = (movies.loc[movies['movieId'].isin(ids)])
#organizar o texto que descreve os gênero dos filmes slecionados
        movie['tokenized_2'] = movie['genres'].apply(lambda x: tokenize(x.lower()))

        movie_grade=pd.DataFrame(list(movie.iloc[:, 3]. values))

        genre_rating = []
        column=np.shape(movie_grade)[1]

#contar os filmes de cada gênero (o filme que tiver n gêneros será contabilizado n vezes)
        for k in range (0, len(genre_all)):
            quantity = 0
            for j in range (0, column):
                selected = movie_grade.loc[(movie_grade[j] == genre_all[k])]
                quantity = quantity+np.shape(selected)[0] 
                
            genre_rating.append(quantity)
#salvar os dados        
        if i == 0:
            np.savetxt('genre_high.dat', genre_rating)
        if i == 1:
            np.savetxt('genre_low.dat', genre_rating)
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
##Agora irei investigar a nota média dada para cada filme e no fim agrupar os filmes por gênero
##Irei primeiramente separar o filmes com o minimo de avaliações ("amount_cut")
##por fim haverá uma segunda divisão, dessa vez considerando a nota média (nesse caso 3)
amount_cut = 500

#montar um data frame com a identidade filme e a quantidade de avaliações de cada filme 
user_rating_amount=(data_table1.groupby(['movieId']).movieId.count().values)
unique=np.sort(list(unique_everseen(data_table1['movieId']))) 
movie_quantity=pd.DataFrame({'movieID': unique, 'quantity': user_rating_amount})

for i in range (0, 2):
    for k in range (0,2):
#selecionar os filmes considerando a quantidade de avaliações        
        if i == 0:
            selected_film = (movie_quantity.loc[movie_quantity['quantity'] < amount_cut])
        
        if i == 1:
            selected_film = (movie_quantity.loc[movie_quantity['quantity'] > amount_cut])

#Selecionar as notas dos filmes selecioanos             
        test = (data_table1.loc[data_table1['movieId'].isin(selected_film.iloc[:,0].values)])
#calcular a média das notas dos filmes selecioandos
        test_2=test.groupby(['movieId']).mean()['rating'].reset_index()
#Separar os filmes pela nota média
        if k == 0:
            selected = (test_2.loc[test_2['rating'] < 3.0])
        if k == 1:
            selected = (test_2.loc[test_2['rating'] > 3.0])
#ler os gêneros dos filmes selecioandos        
        movie_final=(movies.loc[movies['movieId'].isin(selected.iloc[:,0].values)])

        movie_final['tokenized'] = movie_final['genres'].apply(lambda x: tokenize(x.lower()))

        movies_final=pd.DataFrame(list(movie_final.iloc[:, 3]. values))

        genre_rating = []
        column=np.shape(movies_final)[1]
#contar os filmes de cada gênero (o filme que tiver n gêneros será contabilizado n vezes)
        for m in range (0, len(genre_all)):
            quantity = 0
            for n in range (0, column):
                selected = movies_final.loc[(movies_final[n] == genre_all[m])]
                quantity = quantity+np.shape(selected)[0] 
                
            genre_rating.append(quantity)
#Salvar os dados        
        if i == 0:
            if k ==0:
                np.savetxt('low_quantity_low_grade.dat', genre_rating)
            if k ==1:
                np.savetxt('low_quantity_high_grade.dat', genre_rating)    
        if i == 1:
            if k ==0:
                np.savetxt('high_quantity_low_grade.dat', genre_rating)
            if k ==1:
                np.savetxt('high_quantity_high_grade.dat', genre_rating)  
            
        



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
